In [5]:
import os
import json
import textwrap
import random
from openai import OpenAI

# OpenAI API 클라이언트 설정
client = OpenAI(api_key="example")

# ==============================================================================
# 0. 문제 유형별 프롬프트 템플릿 딕셔너리 (TMPLS)
# ==============================================================================
# 지적해주신 핵심 파트입니다.
# 각 문제 유형의 '설계도' 역할을 합니다.

FILL_IN_BLANK_TMPL = """\
[ROLE] 너는 '외국어로서의 한국어' 교재 편집자다. 반드시 JSON만 출력한다.
[GOAL] [INPUT_SENTENCES]에 주어진 문장 중 하나를 활용하여 **주관식 빈칸 채우기** 문제 1개를 만든다.
[INPUT_SENTENCES]
{sentences_bullets}
[INSTRUCTIONS]
- `instruction`: "<보기>와 같이 괄호 안의 단어를 사용하여 문장을 완성하십시오." 와 같은 명확한 지시문을 작성한다.
- `stem_with_blank`: [INPUT_SENTENCES]의 문장 중 하나를 선택하여, 타깃 문법 부분을 빈칸( ___ )으로 바꾼다.
- `hint`: 빈칸에 들어갈 동사/형용사의 기본형을 힌트로 제시한다.
- `example`: 문제에 사용하지 않은 다른 입력 문장 하나를 골라 동일한 형식의 보기 문항을 생성한다.
- **절대로 `options` 필드를 만들지 않는다.**
[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{ "schema_id": "{schema_id}", "format": "fill_in_blank", "input": {{"instruction": "<보기>와 같이 괄호 안의 단어를 사용하여 문장을 완성하십시오.", "example": {{"stem": "나는 점심을 ___ TV를 봤어요. (먹다)", "answer": "먹으면서"}}, "stem_with_blank": "저는 음악을 ___ 공부합니다. ({{hint}})", "hint": "듣다"}}, "answer": {{"completed_sentence": "저는 음악을 들으면서 공부합니다."}}, "rationale": "두 가지 행동을 동시에 함을 나타내는 '-으면서'가 자연스럽습니다. '듣다'는 불규칙 동사이므로 '들으면서'로 활용됩니다."}}"""

MATCH_AND_CONNECT_TMPL = """\
[ROLE] 너는 '외국어로서의 한국어' 교재 편집자다. 반드시 JSON만 출력한다.
[GOAL] 입력된 문장들을 분해하고 재조합하여, 문장 연결하기 문제를 생성한다.
[INPUT_SENTENCES]
{sentences_bullets}
[INSTRUCTIONS]
- `instruction`: "다음 문장을 연결하여 <보기>와 같이 하나의 문장을 만드십시오." 와 같은 지시문을 작성한다.
- `clause_set_A`, `clause_set_B`: 입력된 문장들에서 3~4개를 골라 각각 앞부분과 뒷부분으로 분해하고, 순서를 섞어서 배치한다.
- `example`: **문제에 사용되지 않은 다른 입력 문장 하나를 골라** 분해하여 <보기>를 만든다.
- `answer`: `clause_set_A`의 각 항목에 맞는 `clause_set_B`의 항목을 연결하여 만든 완성 문장들의 배열.
[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{ "schema_id": "{schema_id}", "format": "match_and_connect", "input": {{"instruction": "다음 문장을 연결하여 <보기>와 같이 하나의 문장을 만드십시오.", "example": {{ "clause_A": "아버지는 운동을 하다", "clause_B": "건강을 챙기다", "connected": "아버지는 운동을 하면서 건강을 챙깁니다." }}, "clause_set_A": ["저는 음악을 듣다", "그녀는 친구와 이야기를 나누다", "우리는 여행 계획을 세우다"], "clause_set_B": ["즐거운 시간을 보내다", "공부하다", "웃고 있다"]}}, "answer": {{ "connected_sentences": ["저는 음악을 들으면서 공부합니다.", "그녀는 친구와 이야기를 나누면서 웃고 있습니다.", "우리는 여행 계획을 세우면서 즐거운 시간을 보냈습니다."] }}, "rationale": "조건을 나타내는 '-으면'을 사용하여 앞선 절과 뒷선 절을 자연스럽게 연결할 수 있습니다."}}"""


SENTENCE_CONNECTION_TMPL = """\
[ROLE] 한국어 문장 연결 문제 출제자. JSON만 출력.

[GOAL]
- 입력된 문장 하나를 두 개의 절로 분해하여 문장 연결 문제를 생성한다.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- `instruction`: "다음 두 문장을 <보기>와 같이 목표 문법을 사용하여 한 문장으로 만드십시오." 와 같이 지시문을 작성한다.
- `input`: [INPUT_SENTENCES]의 문장 중 하나를 선택하여 두 개의 독립된 문장(`clause_A`, `clause_B`)으로 분해한다.
- `answer`: 분해되기 전의 원본 문장을 `connected_sentence` 값으로 설정한다.
- `example`: **문제에 사용되지 않은 다른 입력 문장 하나를 골라** 분해하여 <보기>를 만든다.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
  "schema_id": "{schema_id}",
  "format": "sentence_connection",
  "input": {{
    "instruction": "다음 두 문장을 <보기>와 같이 '-(으)면서'를 사용하여 한 문장으로 만드십시오.",
    "example": {{ "clause_A": "나는 점심을 먹습니다.", "clause_B": "TV를 봅니다.", "connected": "나는 점심을 먹으면서 TV를 봅니다." }},
    "clause_A": "저는 음악을 듣습니다.",
    "clause_B": "저는 공부를 합니다."
  }},
  "answer": {{ "connected_sentence": "저는 음악을 들으면서 공부합니다." }},
  "rationale": "두 가지 행동이 동시에 일어남을 나타낼 때 동사 어간에 '-으면서'를 붙여 연결할 수 있습니다."
}}
"""

SENTENCE_CREATION_TMPL = """\
[ROLE] 한국어 문장 생성 문제 출제자. JSON만 출력.

[GOAL]
- 입력된 문장에서 핵심 표현을 추출하여 문장 생성 문제를 만든다.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- `cues`: [INPUT_SENTENCES]의 문장 중 하나에서 핵심이 되는 표현 2~4개를 추출하여 조합할 요소로 제시한다.
- `created_sentence`: `cues`가 추출된 원본 문장을 정답으로 설정한다.
- `example`: **문제에 사용되지 않은 다른 입력 문장 하나에서** 핵심 표현을 추출하여 <보기>를 만든다.
- `instruction`: "<보기>와 같이 주어진 표현을 사용하여 문장을 완성하십시오." 같은 지시문을 작성한다.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
  "schema_id": "{schema_id}",
  "format": "sentence_creation",
  "input": {{
    "instruction": "<보기>와 같이 주어진 표현을 사용하여 '-(으)면서' 문법으로 문장을 완성하십시오.",
    "example": {{ "cues": ["점심을 먹다", "TV를 보다"], "answer": "점심을 먹으면서 TV를 봅니다." }},
    "cues": ["음악을 듣다", "공부하다"]
  }},
  "answer": {{ "created_sentence": "음악을 들으면서 공부합니다." }},
  "rationale": "핵심 표현들을 '-으면서' 문법을 사용하여 자연스러운 문장으로 만들 수 있습니다."
}}
"""

CHOICE_COMPLETION_TMPL = """\
[ROLE] 한국어 문제 출제자. JSON만 출력한다.

[GOAL]
- 목표 문법: {target_grammar}
- 레벨: {level}
- 제시문(prompt)에 맞는 선택지로 문장을 완성하는 문제 1개 생성.

[INSTRUCTIONS]
- `prompt`: 간단한 상황이나 질문을 제시한다.
- `options`: 4개(정답 1, 오답 3)의 선택지를 만든다. 선택지들은 목표 문법의 사용 여부나 정확성으로 정답과 오답이 갈리도록 설계한다.
- `completed_sentence`: `prompt`와 정답 `option`을 자연스럽게 연결한 완성 문장을 만든다.
- `rationale`: 왜 그것이 정답인지 문법적, 문맥적 근거를 설명한다.

---
[COMPLETE_EXAMPLE]
아래는 이 작업을 어떻게 수행해야 하는지에 대한 완벽한 예시다.

## Input Sentences For Example:
- 저는 학교에 가는 길에 친구를 만났어요.
- 퇴근하는 길에 빵을 좀 샀어요.

## Corresponding Output JSON:
{
    "schema_id": "Q_example",
    "format": "choice_completion",
    "input": {
        "prompt": "어제는 정말 바빴어요. 아침 일찍 일어나서 운동을 하고...",
        "options": [
            "회사에 가는 길에 세탁소에 들렀어요.",
            "회사에 가고 세탁소에 들렀어요.",
            "회사에 가려고 세탁소에 들렀어요.",
            "회사에 가지만 세탁소에 들렀어요."
        ]
    },
    "answer": {
        "completed_sentence": "어제는 정말 바빴어요. 아침 일찍 일어나서 운동을 하고 회사에 가는 길에 세탁소에 들렀어요."
    },
    "rationale": "'-는 길에'는 어떤 목적지로 이동하는 도중에 다른 행동을 할 때 사용하는 문법으로, 바쁜 하루의 일과를 설명하는 문맥에 가장 자연스럽습니다."
}
---

[INPUT_SENTENCES]
{sentences_bullets}

[OUTPUT_JSON]
"""


DIALOGUE_COMPLETION_TMPL = """\
[ROLE] 한국어 대화 완성 문제 출제자. JSON만 출력.

[GOAL]
- 목표 문법: {target_grammar}
- 레벨: {level}
- 대화(turn 3개 내외)에서 1곳을 빈칸으로 두고 자연스럽게 채우게 한다.

[INPUT_SENTENCES]
{sentences_bullets}

[INSTRUCTIONS]
- dialogue_with_missing_turns: A/B 대화 배열. 한 턴은 "___" 로 빈칸 표기.
- completed_dialogue: 빈칸을 채운 최종 대화 배열.
- 타깃 문법은 최소 1회 자연스럽게 등장.

[OUTPUT_JSON_SCHEMA_EXAMPLE]
{{
  "schema_id": "{schema_id}",
  "format": "dialogue_completion",
  "input": {{
    "dialogue_with_missing_turns": [
      {{"speaker":"A","text":"..."}},
      {{"speaker":"B","text":"___"}},
      {{"speaker":"A","text":"..."}}
    ]
  }},
  "answer": {{
    "completed_dialogue": [
      {{"speaker":"A","text":"..."}},
      {{"speaker":"B","text":"채워진 문장"}},
      {{"speaker":"A","text":"..."}}
    ]
  }},
  "rationale": "문맥 상의 연결 근거"
}}
"""

TMPLS = {
    "fill_in_blank": FILL_IN_BLANK_TMPL,
    "match_and_connect": MATCH_AND_CONNECT_TMPL,
    "sentence_connection": SENTENCE_CONNECTION_TMPL,
    "sentence_creation": SENTENCE_CREATION_TMPL,
    "choice_completion": CHOICE_COMPLETION_TMPL,
    "dialogue_completion": DIALOGUE_COMPLETION_TMPL,
}

# ==============================================================================
# 1. [1단계] 선택 AI 에이전트
# ==============================================================================

AGENT_PROMPT_TEMPLATE = """\
[ROLE]
당신은 10년차 한국어 교육 과정 설계 전문가입니다. 당신의 임무는 주어진 학습 정보에 기반하여 가장 교육적으로 효과적인 문제 유형을 단 하나만 추천하고, 그 이유를 논리적으로 설명하는 것입니다. 반드시 JSON 형식으로만 출력해야 합니다.
[CONTEXT]
- Target Grammar: {target_grammar}
- Learner Level: {level}
- Available Example Sentences:
{sentences_bullets}
- Available Question Formats:
{formats_bullets}
[INSTRUCTIONS]
다음 3단계의 사고 과정에 따라 최적의 문제 유형을 결정하십시오.
1.  **문법-유형 적합도 분석**: '{target_grammar}' 문법의 핵심 기능은 어떤 문제 유형으로 평가할 때 가장 효과적입니까?
2.  **입력 문장 구조 분석**: 제공된 예문들의 구조적 특징이 어떤 문제 유형을 만들기에 가장 유리합니까?
3.  **학습 목표 및 난이도 고려**: 학습자 레벨({level})을 고려할 때, 어떤 유형이 적절한 학습 효과를 유발할 수 있습니까?
위 분석을 종합하여, 가장 추천하는 문제 유형 하나를 `chosen_format`으로, 그리고 결정 이유를 `rationale`에 서술하여 아래 JSON 스키마에 맞춰 출력하십시오.
[OUTPUT_JSON_SCHEMA]
{{
  "chosen_format": "하나의 문제 유형(string)",
  "rationale": "위 3단계 분석에 기반한 구체적인 선택 이유(string)"
}}"""

def call_llm(prompt: str, model: str = "gpt-4o") -> str:
    """OpenAI 모델을 호출하는 범용 함수"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            response_format={"type": "json_object"}
        )
        return response.choices[0].message.content
    except Exception as e:
        return json.dumps({"error": str(e)})

def bullets(items: list) -> str:
    """리스트를 불렛 포인트 문자열로 변환합니다."""
    return "\n".join(f"- {s}" for s in items)

def select_best_schema(payload: dict) -> dict:
    """최적의 문제 유형을 '결정'하는 AI 에이전트"""
    print("🤖 [1단계] 최적 문제 유형 선택 AI 에이전트를 시작합니다...")
    
    valid_sentences = [item["sentence"] for item in payload["critique_summary"] if item.get("is_valid")]
    available_formats = list(TMPLS.keys())

    prompt = AGENT_PROMPT_TEMPLATE.format(
        target_grammar=payload["target_grammar"],
        level=payload["level"],
        sentences_bullets=bullets(valid_sentences),
        formats_bullets=bullets(available_formats)
    )
    
    print("🧠 선택 LLM을 호출하여 분석 중입니다...")
    raw_json_output = call_llm(prompt) # 에이전트는 고성능 모델 사용 가능
    
    try:
        decision = json.loads(raw_json_output)
        print("✅ 분석 완료!")
        return decision
    except json.JSONDecodeError:
        return {"chosen_format": random.choice(available_formats), "rationale": "AI 에이전트 응답 오류로 랜덤 선택."}


# ==============================================================================
# 2. [2단계] 문제 생성기
# ==============================================================================

def generate_question_item(agent_decision: dict, payload: dict) -> dict:
    """AI 에이전트의 결정을 바탕으로 실제 문제를 '생성'하는 함수"""
    chosen_format = agent_decision.get("chosen_format")
    print(f"\n🚀 [2단계] 선택된 유형 '{chosen_format}'으로 문제 생성을 시작합니다...")

    # 템플릿 딕셔너리에서 선택된 유형의 프롬프트를 가져옴
    template = TMPLS.get(chosen_format)
    
    if not template or template == "...":
        return {"error": f"'{chosen_format}'에 해당하는 프롬프트 템플릿이 정의되지 않았습니다."}
        
    valid_sentences = [item["sentence"] for item in payload["critique_summary"] if item.get("is_valid")]
    
    # 문제 생성을 위한 프롬프트 렌더링
    prompt = template.format(
        sentences_bullets=bullets(valid_sentences),
        schema_id="Q_generated_1", # 실제로는 동적으로 ID 할당
        target_grammar=payload["target_grammar"],
        level=payload["level"]
    )
    
    print("✍️ 생성 LLM을 호출하여 문제 구성 중입니다...")
    raw_json_output = call_llm(prompt) # 생성은 다른 모델 사용 가능

    try:
        generated_question = json.loads(raw_json_output)
        print("✅ 문제 생성 완료!")
        return generated_question
    except json.JSONDecodeError:
        return {"error": "문제 생성 LLM의 응답이 유효한 JSON이 아닙니다."}


# ==============================================================================
# 3. 전체 파이프라인 실행
# ==============================================================================

if __name__ == "__main__":
    example_payload = {
        "level": 3,
        "target_grammar": "-(으)면서",
        "critique_summary": [
            {"sentence": "나는 음악을 들으면서 공부를 하고 있어요.", "is_valid": True, "reason": "OK"},
            {"sentence": "그녀는 책을 읽으면서 차를 마셨습니다.", "is_valid": True, "reason": "OK"},
            {"sentence": "아이들은 놀이터에서 놀면서 친구를 사귀었어요.", "is_valid": True, "reason": "OK"}
        ]
    }

    # --- 1단계 실행 ---
    agent_decision = select_best_schema(example_payload)
    
    print("\n\n" + "="*20, " 1단계: AI 에이전트 결정 ", "="*20)
    print(json.dumps(agent_decision, indent=2, ensure_ascii=False))
    print("="*65)

    # --- 2단계 실행 ---
    if "error" not in agent_decision:
        final_question = generate_question_item(agent_decision, example_payload)
        
        print("\n\n" + "="*20, " 2단계: 최종 생성된 문제 ", "="*20)
        print(json.dumps(final_question, indent=2, ensure_ascii=False))
        print("="*65)
    else:
        print("에이전트 결정 단계에서 오류가 발생하여 문제 생성을 진행하지 않습니다.")

🤖 [1단계] 최적 문제 유형 선택 AI 에이전트를 시작합니다...
🧠 선택 LLM을 호출하여 분석 중입니다...
✅ 분석 완료!


====================  1단계: AI 에이전트 결정  ====================
{
  "chosen_format": "sentence_connection",
  "rationale": "'-(으)면서' 문법의 핵심 기능은 두 가지 동작이나 상태가 동시에 일어나는 것을 표현하는 것입니다. 이를 평가하기 위해서는 두 문장을 연결하여 하나의 복합 문장을 만드는 것이 효과적입니다. 제공된 예문들은 두 가지 동작을 포함하고 있어, 'sentence_connection' 유형을 통해 학습자가 두 동작을 자연스럽게 연결하는 연습을 할 수 있습니다. 또한, 학습자 레벨 3은 중급 수준으로 복합 문장 구조를 이해하고 생성하는 능력을 기르기에 적합하므로, 이 유형이 적절한 학습 효과를 유발할 수 있습니다."
}

🚀 [2단계] 선택된 유형 'sentence_connection'으로 문제 생성을 시작합니다...
✍️ 생성 LLM을 호출하여 문제 구성 중입니다...
✅ 문제 생성 완료!


====================  2단계: 최종 생성된 문제  ====================
{
  "schema_id": "Q_generated_1",
  "format": "sentence_connection",
  "input": {
    "instruction": "다음 두 문장을 <보기>와 같이 '-(으)면서'를 사용하여 한 문장으로 만드십시오.",
    "example": {
      "clause_A": "그녀는 책을 읽었습니다.",
      "clause_B": "그녀는 차를 마셨습니다.",
      "connected": "그녀는 책을 읽으면서 차를 마셨습니다."
    },
    "clause_A": "아이들은 놀이터에서 놀았습니다.",
    "clause_B": "아이들은 친구를 사귀었습니다